<a href="https://colab.research.google.com/github/Koketso-dax/AirBnB_clone/blob/main/Part_III_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part III - Machine Learning Application
### by Koketso Diale

> Now that we are more familiar with the dataset, let us use the insights we have to build a machine learning model to predict a potential Borrower's Prosper Score.


#### Data Gathering

In [19]:
# import packages for data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
# Download Data
!wget
'https://s3.amazonaws.com/udacity-hosted-downloads/ud651/prosperLoanData.csv'

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


'https://s3.amazonaws.com/udacity-hosted-downloads/ud651/prosperLoanData.csv'

In [21]:
# Load in the dataset
loan_df = pd.read_csv('prosperLoanData.csv')
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ListingKey                           113937 non-null  object 
 1   ListingNumber                        113937 non-null  int64  
 2   ListingCreationDate                  113937 non-null  object 
 3   CreditGrade                          28953 non-null   object 
 4   Term                                 113937 non-null  int64  
 5   LoanStatus                           113937 non-null  object 
 6   ClosedDate                           55089 non-null   object 
 7   BorrowerAPR                          113912 non-null  float64
 8   BorrowerRate                         113937 non-null  float64
 9   LenderYield                          113937 non-null  float64
 10  EstimatedEffectiveYield              84853 non-null   float64
 11  EstimatedLoss

> From our analysis in PART I, we have already observed that our data is very imbalanced. Most of the data is from the State of Canada and Texas respectively. Therefore we will need to account for this by randomly selecting limited numbers of Borrowers from each state.

> Machine Learning Frameworks prefer to work with numbers over strings so we will need to use numeric ratings over alpha ratings for simplicity and performance. Additionally some features will need to be Normalized first to work out scaling imbalances amongst the different features.

#### Feature Selection:

In [37]:
# Select suitable features we can use for our models.

features_df = loan_df[[ 'ProsperScore', 'ProsperRating (numeric)',
                       'EmploymentStatus', 'Term','IsBorrowerHomeowner',
                       'IncomeVerifiable', 'IncomeRange',
                       'StatedMonthlyIncome', 'Recommendations']]

> Note:
  * Depending on which rows are useable (complete) and contain fewer outliers, we will then decide on which between `ProsperRating (numeric)` and `ProsperScore` we can use as a target to predict.
  * A similar evaluation will be made for the other features if they are usable as input features.

In [40]:
# define function to determine field with fewest entries.

import pandas as pd

def find_features_with_fewest_entries(dataframe):
  """Finds and prints the dataframe field(s) with the fewest number of entries.
     If a dataframe with no features is given, will print no features found.
     If any object other than a dataframe is given as input, will print an error.

     Arguments:
     ----------
      dataframe : obj
        input pandas dataframe.

      Returns:
      --------
        response : str
          the field(s) with the least number of entries.
    """
  # Check if the input is a DataFrame
  if not isinstance(dataframe, pd.DataFrame):
    print("Input is not a valid DataFrame.")
    return

  # Get the number of rows in the DataFrame
  num_rows = len(dataframe)

  # Initialize variables for min features
  min_entries = float('inf')
  min_feature_names = []

  # Iterate through each column in the DataFrame
  for column in dataframe.columns:
    # Count the number of non-null entries in the column
    num_entries = dataframe[column].count()

  # Check if the current column == current minimum
  if num_entries == min_entries:
    min_feature_names.append(column)
  elif num_entries < min_entries:
    min_entries = num_entries
    min_feature_names = [column]

  # Print the result
  if min_feature_names:
    features_str = ", ".join([f"'{feature}'" for feature in min_feature_names])
    print(f"min features: {features_str} = {min_entries}/{num_rows}")
  else:
    print("No features found in the DataFrame.")

In [41]:
# We can then use this func to outline the features with smallest sample size

find_feature_with_fewest_entries(features_df)


min feature: 'ProsperScore' = 84853/113937


 > From this function we can see that our candidate targets (`ProsperScore` and `ProsperRating (numeric)`) have an equivalent number of entries therefore we may use whichever one of the 2 that translate better via Normalization.